In [5]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
import yfinance as yf
import ipywidgets as widgets
from IPython.display import display, HTML

# Function to create and train the LSTM model
def train_lstm_model(epochs, batch_size, optimizer):
    btc_data = yf.download('BTC-USD', start='2014-09-17', end='2024-12-1')
    btc_data.to_csv('BTC-USD.csv')

    df = pd.read_csv('BTC-USD.csv', header=0)
    df = df.iloc[2:, :]
    df = df.rename(columns={ 'Price' : 'Date'})
    df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    data = df.filter(['Close'])
    dataset = data.values

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(dataset)

    training_data_len = int(np.ceil(len(dataset) * 0.8))

    train_data = scaled_data[0:int(training_data_len), :]

    x_train = []
    y_train = []

    for i in range(60, len(train_data)):
        x_train.append(train_data[i-60:i, 0])
        y_train.append(train_data[i, 0])

    x_train, y_train = np.array(x_train), np.array(y_train)

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))

    model.compile(optimizer=optimizer, loss='mean_squared_error')

    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

    test_data = scaled_data[training_data_len - 60:, :]

    x_test = []
    y_test = dataset[training_data_len:, :]

    for i in range(60, len(test_data)):
        x_test.append(test_data[i-60:i, 0])

    x_test = np.array(x_test)

    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    predictions = model.predict(x_test)
    predictions = scaler.inverse_transform(predictions)

    rmse = np.sqrt(np.mean((predictions - y_test) ** 2))

    train = data[:training_data_len]
    valid = data[training_data_len:]
    valid['Predictions'] = predictions

    return model, scaler, rmse, data, scaled_data

def predict_date(model, scaler, data, date):
    if date not in data.index:
        display(HTML(f"<b>{date} not found in data.</b>"))
        return

    data_for_prediction = data.loc[:date][-60:]
    if len(data_for_prediction) < 60:
        display(HTML(f"Not enough data to make prediction for {date}."))
        return

    scaled_data = scaler.transform(data_for_prediction)
    scaled_data = np.reshape(scaled_data, (1, scaled_data.shape[0], 1))

    future_prediction = model.predict(scaled_data)
    future_prediction = scaler.inverse_transform(future_prediction)[0, 0]

    result_table = pd.DataFrame({
        'Date': [date],
        'Future Prediction': [future_prediction],
        'RMSE': [rmse]
    })

    result_html = result_table.to_html(index=False)
    display(HTML(result_html))

# Iterative Forecasting function
def iterative_forecasting_with_rmse(model, scaler, data, steps, date_range):
    """
    Iterative forecasting with dataset update and RMSE calculation.

    Parameters:
    - model: Trained LSTM model.
    - scaler: MinMaxScaler used during training.
    - data: Original dataset (Pandas DataFrame).
    - steps: Number of steps to predict.
    - date_range: List of future dates for prediction.

    Returns:
    - updated_data: DataFrame with actual and predicted values.
    - rmse: Root Mean Square Error (if actual values are available).
    """
    # Extract the last 60 days of scaled data
    scaled_data = scaler.transform(data['Close'].values.reshape(-1, 1))
    last_known_data = scaled_data[-60:, 0]

    predictions = []
    actuals = []

    for i in range(steps):
        # Prepare input for prediction
        input_reshaped = last_known_data.reshape(1, last_known_data.shape[0], 1)
        prediction = model.predict(input_reshaped, verbose=0)[0, 0]

        predictions.append(prediction)  # Save scaled prediction

        # Update last known data
        last_known_data = np.append(last_known_data[1:], prediction)

        # If actual data is available, use it for RMSE calculation
        if i < len(data) - len(scaled_data):
            actual = scaled_data[len(scaled_data) + i, 0]
            actuals.append(actual)

    # Rescale predictions back to original scale
    predictions_rescaled = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

    # Create a new DataFrame for predictions
    predicted_data = pd.DataFrame({
        'Date': date_range,
        'Predicted Price': predictions_rescaled.flatten()
    })

    # Combine with the original dataset
    updated_data = pd.concat([data, predicted_data.set_index('Date')], axis=0)

    # Calculate RMSE if actual values are available
    if actuals:
        actuals_rescaled = scaler.inverse_transform(np.array(actuals).reshape(-1, 1))
        rmse = np.sqrt(np.mean((actuals_rescaled - predictions_rescaled[:len(actuals)])**2))
    else:
        rmse = None

    return updated_data, rmse


# Create widgets for user input
epochs_widget = widgets.IntText(value=10, description='Epochs:')
batch_size_widget = widgets.IntText(value=32, description='Batch Size:')
optimizer_widget = widgets.Dropdown(options=['adam', 'sgd', 'rmsprop'], value='adam', description='Optimizer:')
date_picker = widgets.DatePicker(description='Select Date')

# Display widgets
display(epochs_widget, batch_size_widget, optimizer_widget, date_picker)

# Button to start training
train_button = widgets.Button(description='Train Model')

def on_train_button_clicked(b):
    global model, scaler, rmse, data, scaled_data
    model, scaler, rmse, data, scaled_data = train_lstm_model(epochs_widget.value, batch_size_widget.value, optimizer_widget.value)

train_button.on_click(on_train_button_clicked)
display(train_button)

# Button to predict
predict_button = widgets.Button(description='Predict')

def on_predict_button_clicked(b):
    selected_date = date_picker.value
    if selected_date:
        selected_date_str = selected_date.strftime('%Y-%m-%d')
        predict_date(model, scaler, data, selected_date_str)
    else:
        display(HTML("<b>Please select a date.</b>"))

predict_button.on_click(on_predict_button_clicked)
display(predict_button)

# Widget for Iterative Forecasting
forecast_days_widget = widgets.IntText(value=90, description='Forecast Days:')
forecast_button = widgets.Button(description='Generate Forecast')

def on_forecast_button_clicked(b):
    global model, scaler, data, scaled_data

    if 'model' not in globals() or 'scaler' not in globals() or 'data' not in globals():
        display(HTML("<b>Model is not trained yet. Train the model first.</b>"))
        return

    forecast_days = forecast_days_widget.value
    if forecast_days < 1 or forecast_days > 90:
        display(HTML("<b>Please enter a valid number of forecast days (1-90).</b>"))
        return

    start_date = data.index[-1] + pd.Timedelta(days=1)
    future_dates = pd.date_range(start_date, periods=forecast_days, freq='D')

    # Perform iterative forecasting with RMSE
    updated_data, rmse = iterative_forecasting_with_rmse(
        model, scaler, data, forecast_days, future_dates
    )

    # Display results
    display(HTML(updated_data.tail(forecast_days).to_html()))
    if rmse is not None:
        display(HTML(f"<b>RMSE for prediction: {rmse:.2f}</b>"))

forecast_button.on_click(on_forecast_button_clicked)

# Display widgets for Iterative Forecasting
display(forecast_days_widget, forecast_button)


IntText(value=10, description='Epochs:')

IntText(value=32, description='Batch Size:')

Dropdown(description='Optimizer:', options=('adam', 'sgd', 'rmsprop'), value='adam')

DatePicker(value=None, description='Select Date')

Button(description='Train Model', style=ButtonStyle())

...

<ipython-input-5-742044c1552b>:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['Predictions'] = predictions


,Close,Predicted Price
Date,,
2024-12-01 00:00:00+00:00,NaN,90636.351562
2024-12-02 00:00:00+00:00,NaN,90427.835938
2024-12-03 00:00:00+00:00,NaN,90028.992188
2024-12-04 00:00:00+00:00,NaN,89504.812500


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


Date,Future Prediction,RMSE
2024-11-29,90489.179688,3077.915094
